In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
plt.style.use('seaborn-whitegrid')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Daten einlesen
data = pd.read_csv("Q:/UZH/FS18/Machine Learning/Projekt/Code/imputed_data.csv")
data = data.set_index(["PERMNO", "DATE"])

data.head(3)

SHRENDDT  BIDLO  ASKHI    PRC         VOL       RET  \
PERMNO DATE                                                                
10107  2006-02-01  20060330.0  26.39  28.04  26.87  11088149.0 -0.042273   
       2006-03-01  20060423.0  26.85  27.89  27.21  14514337.0  0.012653   
       2006-04-01  20060629.0  24.15  27.74  24.15  14689919.0 -0.112459   

                     BID    ASK      SHROUT  ALTPRC     ...      CFACPR_2.0  \
PERMNO DATE                                             ...                   
10107  2006-02-01  26.87  26.88  10333369.0   26.87     ...             0.0   
       2006-03-01  27.24  27.24  10225000.0   27.21     ...             0.0   
       2006-04-01  24.16  24.16  10201203.0   24.15     ...             0.0   

                   CFACPR_4.0  CFACPR_7.0  CFACPR_8.0  CFACSHR_1.0  \
PERMNO DATE                                                          
10107  2006-02-01         0.0         0.0         0.0          1.0   
       2006-03-01         0.0         0.0         0.0          1.0   
       2006-04-01         0.0         0.0         0.0          1.0   

                   CFACSHR_2.0  CFACSHR_4.0  CFACSHR_7.0  CFACSHR_8.0  \
PERMNO DATE                                                             
10107  2006-02-01          0.0          0.0          0.0          0.0   
       2006-03-01          0.0          0.0          0.0          0.0   
       2006-04-01          0.0          0.0          0.0          0.0   

                   PREDICTION  
PERMNO DATE                    
10107  2006-02-01         1.0  
       2006-03-01         0.0  
       2006-04-01         0.0  

[3 rows x 305 columns]

In [3]:
# Extract labels
feat_labels = data.columns[:-1]

In [4]:
# Assign features & response to X and y (p.117)
X = data.drop(["PRC_RET", "PREDICTION", "PRC_MOV"], axis=1)
y = data['PREDICTION']
display(y.head())

PERMNO  DATE      
10107   2006-02-01    1.0
        2006-03-01    0.0
        2006-04-01    0.0
        2006-05-01    1.0
        2006-06-01    1.0
Name: PREDICTION, dtype: float64

In [5]:
# Train/Test split (p. 104ff) 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state =0, stratify =y)

In [6]:
# Variablen standardisieren
sc = StandardScaler ()
X_train_std =  sc.fit_transform(X_train )
X_test_std = sc.transform(X_test)

In [7]:
# Feature selection (p. 146) Skript
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state =1)
forest.fit(X_train_std , y_train)
importances = forest.feature_importances_   

In [9]:
# sortieren
indices = np.argsort(importances)[::-1]
i = 0
n = 15
for i in range(n):
    print('{0:2d} {1:7s} {2:6.4f}'.format(i+1, feat_labels[indices[i]], importances[indices[i]]))

 1 ewretx  0.0283
 2 vwretx  0.0277
 3 RETX    0.0254
 4 ewretd  0.0249
 5 sprtrn  0.0236
 6 VOL     0.0236
 7 vwretd  0.0231
 8 RET     0.0229
 9 SHRENDDT 0.0194
10 ALTPRC  0.0193
11 ALTPRCDT 0.0189
12 PEG_trailing 0.0188
13 SHROUT  0.0186
14 staff_sale 0.0180
15 ASKHI   0.0177
